<a href="https://colab.research.google.com/github/Im-jDiOt/DeepLearning-Term-Proj/blob/feature-resnet/1%EC%B0%A8%EC%8B%A4%ED%96%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm  # timm 라이브러리 사용
from sklearn.model_selection import KFold

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

# --- 1. 데이터 로드 및 하이퍼파라미터 ---
base_dir = r'/content/drive/MyDrive/Colab Notebooks'
# [수정] 'data' 폴더 제거
driver_csv_path = os.path.join(base_dir, 'driver_imgs_list.csv')
# [수정] 'data/imgs' 폴더 제거
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# [수정] ResNet-50의 표준 입력 크기는 224x224 입니다.
img_size = 224
batch_size = 32
num_classes = 10
num_epochs = 50
learning_rate = 0.001
num_workers = 0

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")
print(f"이미지 크기: {img_size}x{img_size}")
print(f"배치 크기: {batch_size}")

driver_df = pd.read_csv(driver_csv_path)
print(f"고유 운전자 수: {driver_df['subject'].nunique()}명")

# --- 2. 5-Fold 교차 검증 (운전자 기준 분할) ---
all_drivers = sorted(driver_df['subject'].unique())
n_folds = 5
kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)

fold_splits = []
print("K-Fold 운전자 분할:")
for fold_idx, (train_indices, val_indices) in enumerate(kfold.split(all_drivers)):
    train_drivers = [all_drivers[i] for i in train_indices]
    val_drivers = [all_drivers[i] for i in val_indices]

    fold_splits.append({
        'fold': fold_idx+1,
        'train_drivers': train_drivers,
        'val_drivers': val_drivers
    })

    # (로그 출력은 간결하게)
    train_imgs = driver_df[driver_df['subject'].isin(train_drivers)]
    val_imgs = driver_df[driver_df['subject'].isin(val_drivers)]
    print(f"  Fold {fold_idx+1}: 학습 {len(train_imgs)}개 | 검증 {len(val_imgs)}개")


# --- 3. 커스텀 데이터셋 (DriverDataset) ---
class DriverDataset(Dataset):
    """운전자 행동 데이터셋"""

    def __init__(self, data_dir, driver_df, driver_list, transform=None, is_test=False):
        self.data_dir = data_dir
        self.transform = transform
        self.is_test = is_test
        self.images = []
        self.labels = []

        if is_test:
            test_images_dir = data_dir
            # (OS Error 방지를 위해 os.listdir 후 .endswith 체크)
            try:
                for img_name in os.listdir(test_images_dir):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.images.append(os.path.join(test_images_dir, img_name))
            except Exception as e:
                print(f"테스트 디렉토리 읽기 오류: {e}")
        else: #is_train
            driver_subset = driver_df[driver_df['subject'].isin(driver_list)]

            for _, row in driver_subset.iterrows():
                class_name = row['classname']
                img_name = row['img']
                img_path = os.path.join(data_dir, class_name, img_name)
                self.images.append(img_path)
                class_idx = int(class_name[1:])
                self.labels.append(class_idx)

        print(f"{'테스트' if is_test else '운전자 ' + str(len(driver_list))+'명'}, 데이터 {len(self.images)}개 이미지 로드")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"이미지 로드 오류: {img_path}, {e}")
            return None, None # (오류 발생 시 None 반환)

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, os.path.basename(img_path)
        else:
            label = self.labels[idx]
            return image, label

# --- 4. 데이터 전처리 (Transforms) ---
# (팀원분의 'team_transform' 파이프라인 그대로 사용)
team_transform_train = transforms.Compose([
	transforms.Resize((img_size, img_size)),
	transforms.RandomRotation(degrees=15),
	transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.05),
	transforms.ToTensor(),
	transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

team_transform_eval = transforms.Compose([
	transforms.Resize((img_size, img_size)),
	transforms.ToTensor(),
	transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

# --- 5. 학습/검증 헬퍼 함수 ---
def train_epoch(model, train_loader, criterion, optimizer, device):
    """ 한 에폭(Epoch) 동안 모델을 학습시킵니다. """
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        # (데이터셋 오류 시 배치 스킵)
        if inputs is None or labels is None:
            continue

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    if total_samples == 0:
        return 0.0, 0.0

    epoch_loss = running_loss / total_samples
    epoch_acc = (correct_predictions.double() / total_samples) * 100
    return epoch_loss, epoch_acc.item()

def validate(model, val_loader, criterion, device):
    """ 검증 데이터셋을 이용해 모델의 성능을 평가합니다. """
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validating"):
            if inputs is None or labels is None:
                continue

            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

    if total_samples == 0:
        return 0.0, 0.0

    epoch_loss = running_loss / total_samples
    epoch_acc = (correct_predictions.double() / total_samples) * 100
    return epoch_loss, epoch_acc.item()


# --- 6. 핵심 훈련 함수 (train_fold) ---
def train_fold(fold_idx, train_drivers, val_drivers):
    """한 폴드 학습 (Log Loss 기준)"""

    print(f"\n========== Fold {fold_idx}/{n_folds} ==========")

    train_dataset = DriverDataset(
        train_dir, driver_df, train_drivers,
        transform=team_transform_train, is_test=False
    )
    val_dataset = DriverDataset(
        train_dir, driver_df, val_drivers,
        transform=team_transform_eval, is_test=False
    )

    # (DataLoader에서 오류 배치를 스킵하기 위한 collate_fn)
    def collate_fn(batch):
        batch = list(filter(lambda x: x[0] is not None, batch))
        if not batch: return torch.Tensor(), torch.Tensor()
        return torch.utils.data.dataloader.default_collate(batch)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, collate_fn=collate_fn
    )

    print(f"학습 배치 수: {len(train_loader)}")
    print(f"검증 배치 수: {len(val_loader)}")

    # --- [모델 수정] Inception-v4 -> ResNet-50 ---
    model = timm.create_model(
        'resnet50',     # timm에서 ResNet-50 모델 사용
        pretrained=True,
        num_classes = num_classes
    )
    model = model.to(device)

    # (팀원분의 훈련 방식(Adam + ReduceLROnPlateau) 그대로 사용)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=7
    )

    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }

    best_val_loss = float('inf')
    # [수정] 모델 이름 변경
    best_model_path = f'best_resnet50_fold{fold_idx}.pth'

    # 에폭 학습
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 70)

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(f'\n📊 Epoch {epoch+1} 결과:')
        print(f'  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%')
        print(f'  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'fold': fold_idx,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_acc': val_acc,
                'history': history
            }, best_model_path)
            print(f'  ✓ 최고 성능 모델 저장! (Val Loss: {val_loss:.4f})')

    print(f"\n✓ Fold {fold_idx} 학습 완료! 최저 검증 손실: {best_val_loss:.4f}")

    return {
        'fold': fold_idx,
        'history': history,
        'best_val_loss': best_val_loss,
        'best_val_acc': max(history['val_acc']),
        'model_path': best_model_path
    }

# --- 7. 메인 실행 (Fold 2만 실행) ---
all_fold_results = []
try:
    # (팀원분의 전략대로 Fold 2 (인덱스 1)만 우선 실행)
    fold_info = fold_splits[0]
    fold_idx = fold_info['fold']
    train_drivers = fold_info['train_drivers']
    val_drivers = fold_info['val_drivers']

    fold_result = train_fold(fold_idx, train_drivers, val_drivers)
    all_fold_results.append(fold_result)

    torch.cuda.empty_cache()

    print(f"\n--- 최종 결과 (Fold {fold_result['fold']}) ---")
    print(f"  최저 Val Loss: {fold_result['best_val_loss']:.4f}")
    print(f"  최고 Val Acc: {fold_result['best_val_acc']:.2f}%")

except Exception as e:
    print(f"\n🚨 훈련 중 오류 발생: {e}")


# --- 8. 단일 모델 예측 및 제출 ---
if not all_fold_results:
    print("🚨 오류: 학습된 폴드 결과가 없어 예측을 생략합니다.")
else:
    result = all_fold_results[0] # (Fold 2의 결과)
    fold_idx = result['fold']
    model_path = result['model_path']

    print("\n" + "=" * 70)
    print(f"🔮 Fold {fold_idx} 단일 모델 예측 시작")
    print("=" * 70)

    test_dataset = DriverDataset(
        test_dir, driver_df, [],
        transform=team_transform_eval, is_test=True
    )

    def collate_fn_test(batch):
        batch = list(filter(lambda x: x[0] is not None, batch))
        if not batch: return torch.Tensor(), []
        images, filenames = zip(*batch)
        return torch.stack(images), list(filenames)

    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, collate_fn=collate_fn_test
    )

    print(f"테스트 샘플: {len(test_dataset)}개")

    # --- [모델 수정] ResNet-50 뼈대 생성 ---
    model = timm.create_model('resnet50', pretrained=False, num_classes=num_classes)

    print(f"\n📁 모델 로드 중: {model_path}...")
    try:
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Fold {fold_idx} 모델 (Epoch {checkpoint['epoch']+1}, Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}) 로드 완료.")
    except Exception as e:
        print(f"🚨 모델 로드 실패: {e}")
        raise # (오류 발생 시 중지)

    model = model.to(device)
    model.eval()

    predictions = []
    img_names = []

    with torch.no_grad():
        for images, filenames in tqdm(test_loader, desc=f'Fold {fold_idx} 예측'):
            if images.nelement() == 0: continue # (빈 배치 스킵)

            images = images.to(device)
            outputs = model(images)

            probs = torch.softmax(outputs, dim=1)
            predictions.append(probs.cpu().numpy())
            img_names.extend(filenames)

    final_predictions = np.vstack(predictions)
    print(f"\n✓ 예측 완료: {final_predictions.shape}")

    # --- Submission 파일 생성 ---
    class_cols = [f'c{i}' for i in range(num_classes)]
    submission_data = {'img': img_names}
    for i, col in enumerate(class_cols):
        submission_data[col] = final_predictions[:, i]

    submission = pd.DataFrame(submission_data)

    # [수정] 파일 이름 변경
    submission_file = f'resnet50_fold{fold_idx}_single_model_submission.csv'
    submission.to_csv(submission_file, index=False)

    print("\n" + "=" * 70)
    print(f"✅ Submission 파일 생성 완료: {submission_file}")
    print(f"✅ 총 {len(submission)}개 이미지 예측")
    print("=" * 70)
    print("\n📋 Submission 샘플:")
    print(submission.head())